In [1]:
import os

In [2]:
%pwd

'c:\\Users\\adminn\\Documents\\GitHub\\NBA\\research'

In [3]:
os.chdir('../')


In [43]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:

    root_dir: Path
    source: Path
    local_data_file:Path


In [44]:
from NBA_Project.constants import *
from src.NBA_Project.utils.common import read_yaml,create_directories


In [45]:
class ConfigurationManager:

    def __init__(self,
                 config_filepath=CONFIG_FILE_PATH,
                 params_filepath=PARAMS_FILE_PATH,
                 schema_filepath=SCHEMA_FILE_PATH):
        
        self.config=read_yaml(config_filepath)
        self.params=read_yaml(params_filepath)
        self.schema=read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self)->DataIngestionConfig:

        config=self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config=DataIngestionConfig(
            root_dir=config.root_dir,
            source=config.source,
            local_data_file=config.local_data_file,
        )

        return data_ingestion_config


In [46]:
import os
import urllib.request as request
import zipfile
from NBA_Project import logger
from NBA_Project.utils.common import get_size
import shutil



In [47]:
class DataIngestion:

    def __init__(self,config: DataIngestionConfig):

        self.config=config

    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            source_path = self.config.source
            destination_path = self.config.local_data_file

            print(source_path)

            if os.path.exists(source_path) and os.path.isfile(source_path):
                try:
                    shutil.copy(source_path, destination_path)
                    logger.info(f"File copied from {source_path} to {destination_path}")
                except Exception as e:
                    logger.error(f"Failed to copy the file: {str(e)}")
            else:
                logger.error("Source file does not exist or is not a regular file.")
        else:
            logger.info(f"File already exists of size {os.path.getsize(self.config.local_data_file)}")


    def extrac_zip_file(self):

        unzip_path=self.config.unzip_dir

        os.makedirs(unzip_path,exist_ok=True)

        with zipfile.ZipFile(self.config.local_data_file,'r') as zip_ref:

            zip_ref.extractall(unzip_path)

In [49]:
try: 
    config=ConfigurationManager()
    data_ingestion_config=config.get_data_ingestion_config()
    data_ingestion=DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
except Exception as e:

    raise e

[2023-10-19 21:10:35,497: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-10-19 21:10:35,499: INFO: common: yaml file: params.yaml loaded successfully]
[2023-10-19 21:10:35,500: INFO: common: yaml file: schema.yaml loaded successfully]
[2023-10-19 21:10:35,502: INFO: common: created directory at artifacts]
[2023-10-19 21:10:35,504: INFO: common: created directory at artifacts/data_ingestion]
data\nba_logreg.csv
[2023-10-19 21:10:35,507: INFO: 3135120561: File copied from data\nba_logreg.csv to artifacts/data_ingestion/data.csv]
